In [1]:
#importer

# aktiver venv
#.\.venv\Scripts\activate

# Installer pakker
# pip install -r ./postgis_til_parquet/requirements.txt

import duckdb                   # SQL-database som kvakker raskere enn andre databaser
import requests, json           # Sender brev til internett og får svar i pene bokser
import numpy as np              # Matematikk på steroider - for når du vil regne fort
import keplergl                 # Gjør kart så fancy at selv Galileo ville blitt imponert
import geopandas as gpd         # Som pandas (det populære dataanalysebiblioteket for tabelldata, ikke bambusbærene), men med retningssans og kart
from dotenv import load_dotenv  # Hemmelighetsbevarer for passord og API-nøkler
import os                       # For når du må snakke med datamaskinen på dens eget språk (for å hente hemmelighetene våre fra .env)

# Last inn hemmeligheter
load_dotenv()

True

In [2]:
# Kom i gang med DuckDB i minnet

# Start kvakebassen
con = duckdb.connect(database=':memory::') # :memory:: betyr at vi bruker minnet, ikke en fil på disken. Det er raskere og mer praktisk for midlertidige data.


# Last inn noen tilleggsverktøy i DuckDB
con.execute("INSTALL spatial;")  # Gir DuckDB retningssans så den kan finne frem i geografigrøten
con.execute("LOAD spatial;")     

con.execute("INSTALL httpfs;")   # Lærer anden å surfe på internett for å hente data fra skyen
con.execute("LOAD httpfs;")      

con.execute("INSTALL postgres;") # Ansetter en tolk som snakker flytende PostgreSQL
con.execute("LOAD postgres;")    

In [3]:
# Agder er fylke med nr 42, for hele listen: "https://api.test.kartverket.no/kommuneinfo/v1/fylker"
# 42 - også kjent som "svaret på livet, universet og alt mulig" - passende for et flott fylke!
agder_req = requests.get("https://api.test.kartverket.no/kommuneinfo/v1/fylker/42/omrade")
agder_json = agder_req.json()  # Pakker ut JSON
agder_geojson_str = json.dumps(agder_json['omrade'])  # Konverterer til streng - fordi datamaskiner er kresne på hva dem spiser

# Alternativ for Kristiansand kommune

# Kristiansand er kommune med nr 4204, for hele listen: "https://api.test.kartverket.no/kommuneinfo/v1/kommuner"
# Koden under er på ferie - kommentert ut for å nyte solen
#krs_req = requests.get("https://api.test.kartverket.no/kommuneinfo/v1/kommuner/4204/omrade")
#krs_json = krs_req.json()
#krs_geojson_str = json.dumps(krs_json['omrade'])


# BBOX koordinater - fordi vi liker å tenke innenfor boksen
min_x = con.execute("SELECT ST_XMin(ST_GeomFromGeoJSON(?)) as min_x", [agder_geojson_str]).fetchone()[0]  # Finner vestligste punkt - der sola går til ro
min_y = con.execute("SELECT ST_YMin(ST_GeomFromGeoJSON(?)) as min_y", [agder_geojson_str]).fetchone()[0]  # Finner sørligste punkt - nesten Danmark!
max_x = con.execute("SELECT ST_XMax(ST_GeomFromGeoJSON(?)) as max_x", [agder_geojson_str]).fetchone()[0]  # Finner østligste punkt - morgenfrisk soloppgang
max_y = con.execute("SELECT ST_YMax(ST_GeomFromGeoJSON(?)) as max_y", [agder_geojson_str]).fetchone()[0]  # Finner nordligste punkt - der trollene bor

# Bounding box: 6.14970490204483, 57.759005282950625 to 9.668876552798709, 59.67268697626103

print(f"Bounding box: {min_x}, {min_y} to {max_x}, {max_y}")  # Skriver ut Agders hjørner


Bounding box: 6.14970490204483, 57.759005282950625 to 9.668876552798709, 59.67268697626103


# Finn det største bygget i Agder
 Vi bruker Overture Maps' detaljerte bygningsdata og Kartverkets polygon for Agder for å avsløre noen store bygg i fylket.

### Spørmål å gruble på 🤔
* Hva tror du er det største bygget i Agder målt i areal?
* Hvor stort tror du det største boligbygget (residential) er i kvadratmeter?

In [4]:

# Her ber vi databasen om å finne alle byggene i Agder og sortere dem etter størrelse.
# Over 2 milliarder byggninger

query_hent_bygg = f"""
COPY(
SELECT
  names.primary as navn,
  ROUND(ST_Area(ST_Transform(geometry, 'EPSG:4326', 'EPSG:25832')), 2) as area_m2,
  class,
  geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*',
hive_partitioning=true)
WHERE bbox.xmin BETWEEN {min_x} AND {max_x}
  AND bbox.ymin BETWEEN {min_y} AND {max_y}
ORDER BY area_m2 DESC
) TO 'agders_50_storste_bygg.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000);
"""

#con.execute(query_hent_bygg)

In [5]:
# Litt visualisering

# Vis resultat fra parquet filen: top 5 bygg med størst areal
con.sql("SELECT navn, area_m2 FROM read_parquet('agders_50_storste_bygg.parquet') WHERE navn IS NOT NULL LIMIT 5").show()

# Vis resultat fra parquet filen: top 5 bygg med størst areal og klasse = residential
con.sql("SELECT * EXCLUDE (geometry, class) FROM read_parquet('agders_50_storste_bygg.parquet') WHERE class = 'residential' LIMIT 5").show()


# Tegn kepplergl kart
# visualisere dataene på et kart
# Lager et interaktivt kart som viser plasseringen av de største byggene

# Henter de 5 største byggene med navn for kartvisning
top_5_bygg_df = con.sql("""
    SELECT * EXCLUDE (geometry),
    ST_AsText(geometry) as geometry_wkt
    FROM read_parquet('agders_50_storste_bygg.parquet') 
    WHERE navn IS NOT NULL 
    ORDER BY area_m2 DESC
    LIMIT 5
""").df()

# Konverterer dataene til geospatial format for kartvisning
top_5_bygg_gdf = gpd.GeoDataFrame(top_5_bygg_df, geometry=gpd.GeoSeries.from_wkt(top_5_bygg_df['geometry_wkt'].tolist()),crs="EPSG:4326")

# Trenger ikke WKT kolonnen lenger, så vi kaster den
top_5_bygg_gdf = top_5_bygg_gdf.drop(columns=['geometry_wkt'])

# Oppretter et interaktivt kart der vi kan utforske bygningene
map = keplergl.KeplerGl(height=600)

# Legger til dataene på kartet så vi kan se hvor de største byggene befinner seg
map.add_data(data=top_5_bygg_gdf, name="Top 5 største bygg i Agder")
map

┌──────────────────┬───────────┐
│       navn       │  area_m2  │
│     varchar      │  double   │
├──────────────────┼───────────┤
│ Sørlandssenteret │ 207172.25 │
│ Nordan           │ 125469.25 │
│ Herkules         │  93700.21 │
│ Chassix Norway   │  81781.28 │
│ Eramet           │  80619.32 │
└──────────────────┴───────────┘

┌─────────┬─────────┐
│  navn   │ area_m2 │
│ varchar │ double  │
├─────────┼─────────┤
│ NULL    │ 10474.3 │
│ NULL    │ 5303.03 │
│ NULL    │ 4830.56 │
│ NULL    │ 4814.44 │
│ NULL    │ 4064.66 │
└─────────┴─────────┘

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Top 5 største bygg i Agder': {'index': [0, 1, 2, 3, 4], 'columns': ['navn', 'area_m2', 'class'…

# Finn mest “scenic route” (innenfor 50meter fra kystlinje / hav) Agder

* Vi skal ut på sykkeltur og vil gjerne se hvilke småveier det er mulig å sykle eller gå
* Henter ut kystlinje data fra PostgreSQL base i Norkarts datavarehus
* Henter ut veg segment data fr Overturemaps
* Gjennomfører en analyse mellom disse (50 m avstand) for å finne veier med utsikt, langs sjøkanten. 


In [6]:
# Hent noen hemmeligheter fra .env-filen
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')



# Leser data fra postgresql
# Avlaster henting av data til Postgresql, DuckDB skriver resultatet til parquet-fil
query_hent_vannkanter_agder = f"""
ATTACH 'dbname={db_name} user={db_user} password={db_pass} host={db_host} port={db_port}' 
  AS postgres_db (TYPE postgres, READ_ONLY);

COPY (
  SELECT * FROM postgres_query('postgres_db', '
  
SELECT 
    dvh_id as id,
    ST_AsBinary(ST_Force2D(ST_Transform(geom, 4326))) as geometry
FROM fkb50.v_kommunene_104_kystkontur
WHERE ST_Intersects(
    geom, 
    ST_Transform(ST_MakeEnvelope({min_x}, {min_y}, {max_x}, {max_y}, 4326), 3857)
)
  ')

) TO 'agder_vannkanter.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000);

DETACH postgres_db;
"""

# koble fra basen om nødvendig
try:
    con.execute("DETACH postgres_db;")
except:
    pass

#con.execute(query_hent_vannkanter_agder)


In [7]:
# Hent veier i Agder


# Henter ut veg segmeneter i Agder med klasser som er relevante for sykling og gangveier
query_hent_vei = f"""
COPY(
SELECT
    id,
    names.primary as name,
    class,
    geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=transportation/type=segment/*',
hive_partitioning=true)
WHERE bbox.xmin BETWEEN {min_x} AND {max_x}
  AND bbox.ymin BETWEEN {min_y} AND {max_y}
  AND class IN ('path', 'cycleway', 'footway', 'residential', 'living_street', 'track', 'service')
) TO 'agder_sykkelveier.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000);
"""

#con.execute(query_hent_vei)


In [8]:
# Bruker sykkelvei data og vannkanter for å finne veier som er 50m eller nærmere vannkanten
# Konverterer geometrien til EPSG:25832 for å ha riktige avstandsberegninger i meter
# Bruker indekser for å gjøre spørringen raskere
# ST_DWithin er en funksjon som sjekker om to geometriske objekter er innenfor en viss avstand fra hverandre

query_nydelige_kystsykkelveier= """CREATE TABLE IF NOT EXISTS sykkelveier_transformed AS 
SELECT 
    id, name, class, geometry,
    ST_Transform(geometry, 'EPSG:4326', 'EPSG:25832') AS geometry_25832
FROM read_parquet('agder_sykkelveier.parquet');

CREATE TABLE IF NOT EXISTS vannkanter_transformed AS 
SELECT 
    *,
    ST_Transform(ST_GeomFromWKB(geometry), 'EPSG:4326', 'EPSG:25832') AS geometry_25832
FROM read_parquet('agder_vannkanter.parquet');

CREATE INDEX IF NOT EXISTS idx_sykkelveier_geom ON sykkelveier_transformed USING RTREE (geometry_25832);
CREATE INDEX IF NOT EXISTS idx_vannkanter_geom ON vannkanter_transformed USING RTREE (geometry_25832);

COPY(
    SELECT DISTINCT
        v.id as vei_id,
        v.name as vei_name,
        v.class as vei_class,
        ROUND(ST_Length(v.geometry_25832), 2) as length_m,
        v.geometry
    FROM 
        sykkelveier_transformed v,
        vannkanter_transformed k
    WHERE 
        ST_DWithin(v.geometry_25832, k.geometry_25832, 50)
    ORDER BY 
        length_m DESC
) TO 'agder_nydelige_sykkelveier.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000);
"""

#con.execute(query_nydelige_kystsykkelveier)

In [9]:
# visualisering av sykkelveier fra analysen med keplergl


map = keplergl.KeplerGl(height=1000)


sykkelveier_df = con.sql("""
    SELECT * EXCLUDE (geometry),
    ST_AsText(geometry) as geometry_wkt
    FROM read_parquet('agder_nydelige_sykkelveier.parquet')
""").df()

sykkelveier_df = gpd.GeoDataFrame(sykkelveier_df, geometry=gpd.GeoSeries.from_wkt(sykkelveier_df['geometry_wkt'].tolist()),crs="EPSG:4326")
sykkelveier_df = sykkelveier_df.drop(columns=['geometry_wkt'])

map.add_data(data=sykkelveier_df, name="Nydelige veier og stier man kan sykle langs sjøen i Agder")

map


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Nydelige veier og stier man kan sykle langs sjøen i Agder': {'index': [0, 1, 2, 3, 4, 5, 6, 7,…

# Finn eiendommen med mest myr i Agder

* Bekymret for bomkjøp
* Opptatt av å bevare natur?

In [10]:

# Hent teiger og myr i Agder fra to forskjellige databaser i Norkarts Datavareshus
db_host2 = os.environ.get('DB_HOST2')
db_port2 = os.environ.get('DB_PORT2')
db_name2 = os.environ.get('DB_NAME2')
db_user2 = os.environ.get('DB_USER2')
db_pass2 = os.environ.get('DB_PASS2')


query_hent_teig_agder = f"""
ATTACH 'dbname={db_name2} user={db_user2} password={db_pass2} host={db_host2} port={db_port2}' 
  AS postgres_db (TYPE postgres, READ_ONLY);

COPY (
  SELECT * FROM postgres_query('postgres_db', '
  
SELECT 
    dvh_id as id,
    ST_AsBinary(ST_Force2D(ST_Transform(geom, 4326))) as geometry
FROM nmk.v_kommunene_91_teig
WHERE ST_Intersects(
    geom, 
    ST_Transform(ST_MakeEnvelope({min_x}, {min_y}, {max_x}, {max_y}, 4326), 3857)
)
  ')

) TO 'agder_eiendommer.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 50_000);

DETACH postgres_db;
"""

query_hent_myr_agder = f"""
ATTACH 'dbname={db_name} user={db_user} password={db_pass} host={db_host} port={db_port}' 
  AS postgres_db (TYPE postgres, READ_ONLY);

COPY (
  SELECT * FROM postgres_query('postgres_db', '
  
SELECT 
    dvh_id as id,
    ST_AsBinary(ST_Force2D(ST_Transform(geom, 4326))) as geometry
FROM fkb50.v_kommunene_104_arealressursflate
WHERE arealtype = 60::VARCHAR
  AND ST_Intersects(
    geom, 
    ST_Transform(ST_MakeEnvelope({min_x}, {min_y}, {max_x}, {max_y}, 4326), 3857)
)
  ')

) TO 'agder_myr.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 50_000);

DETACH postgres_db;
"""

# koble fra basen om nødvendig
try:
    con.execute("DETACH postgres_db;")
except:
    pass

#con.execute(query_hent_teig_agder)
#con.execute(query_hent_myr_agder)

In [11]:
# overlap mellom eiendommer og myr
query_analyse_myr_eiendommer = f"""
DROP TABLE IF EXISTS teig_transformed; DROP TABLE IF EXISTS myr_transformed;
CREATE TABLE teig_transformed AS
SELECT 
    id as teig_id,
    ST_GeomFromWKB(geometry) AS geometry,
    -- Transform to a projected coordinate system for accurate area measurements
    ST_Transform(ST_GeomFromWKB(geometry), 'EPSG:4326', 'EPSG:25832') AS geometry_25832
FROM read_parquet('agder_eiendommer.parquet');

CREATE TABLE myr_transformed AS
SELECT 
    id as myr_id,
    ST_GeomFromWKB(geometry) AS geometry,
    -- Transform to the same coordinate system as teig
    ST_Transform(ST_GeomFromWKB(geometry), 'EPSG:4326', 'EPSG:25832') AS geometry_25832
FROM read_parquet('agder_myr.parquet');

-- Indekser for hastighet!
CREATE INDEX idx_teig_geom ON teig_transformed USING RTREE (geometry_25832);
CREATE INDEX idx_myr_geom ON myr_transformed USING RTREE (geometry_25832);

COPY(
    SELECT 
        t.teig_id,
        COUNT(m.myr_id) AS num_myr_overlaps,
        ROUND(SUM(ST_Area(ST_Intersection(t.geometry_25832, m.geometry_25832))), 2) AS total_myr_area_m2,
        -- BEregn areal av eiendommene i m2
        ROUND(SUM(ST_Area(ST_Intersection(t.geometry_25832, m.geometry_25832))) / 
              MIN(ST_Area(t.geometry_25832)) * 100, 2) AS myr_percentage,
        -- hent ut geometrien til eiendommene for å tegne dem i kartet
        ANY_VALUE(t.geometry) AS property_geometry
    FROM 
        teig_transformed t
    JOIN 
        myr_transformed m
    ON 
        ST_Intersects(t.geometry_25832, m.geometry_25832)
    GROUP BY 
        t.teig_id
    ORDER BY 
        total_myr_area_m2 DESC
    LIMIT 10
) TO 'agder_eiendommer_med_mest_myr.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000);
"""

#con.execute(query_analyse_myr_eiendommer)

In [12]:

# Eiendommer med mest myr i Agder
con.sql("""
    SELECT 
        teig_id, 
        num_myr_overlaps, 
        total_myr_area_m2, 
        myr_percentage
    FROM read_parquet('agder_eiendommer_med_mest_myr.parquet')
    ORDER BY total_myr_area_m2 DESC
""").show()


# Visualize in KeplerGL
map_myr = keplergl.KeplerGl(height=1000)

# Prepare data for KeplerGL using GeoPandas
myr_eiendommer_df = con.sql("""
    SELECT 
        teig_id, 
        num_myr_overlaps, 
        total_myr_area_m2, 
        myr_percentage,
        ST_AsText(property_geometry) AS geometry_wkt
    FROM read_parquet('agder_eiendommer_med_mest_myr.parquet')
    ORDER BY total_myr_area_m2 DESC
""").df()

myr_eiendommer_gdf = gpd.GeoDataFrame(
    myr_eiendommer_df, 
    geometry=gpd.GeoSeries.from_wkt(myr_eiendommer_df['geometry_wkt'].tolist()),
    crs="EPSG:4326"
)
myr_eiendommer_gdf = myr_eiendommer_gdf.drop(columns=['geometry_wkt'])

# Add data to the map
map_myr.add_data(data=myr_eiendommer_gdf, name="Eiendommer med mest myr i Agder")

# Display the map
map_myr

┌──────────────────────────────────────┬──────────────────┬───────────────────┬────────────────┐
│               teig_id                │ num_myr_overlaps │ total_myr_area_m2 │ myr_percentage │
│                 uuid                 │      int64       │      double       │     double     │
├──────────────────────────────────────┼──────────────────┼───────────────────┼────────────────┤
│ f9642a1f-d448-48f8-b2c1-d65dd92f59ff │              525 │       22653530.87 │           7.89 │
│ f59ebc37-80d1-4e84-a2f4-ae152b25be27 │              331 │       14737388.64 │          16.47 │
│ c2e51e4f-064d-48aa-8b57-aae8710135bb │              592 │       13955835.53 │           7.17 │
│ 3fd91622-3d4f-4d2f-9768-0dcf141776ff │               55 │       12055805.65 │          16.24 │
│ 84e756f8-6066-4b23-81e5-0ee660fb2be7 │              243 │       10877595.01 │          13.24 │
│ 17557ecd-67bd-40cc-87a8-efdfbb9c71fc │              134 │       10291446.24 │          10.78 │
│ 70c9df1c-abd2-4053-a3a7-1bf7

KeplerGl(data={'Eiendommer med mest myr i Agder': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'columns': ['teig_…

# Alle puber i Agder og hvilke er nærmest en kirke?

* på sørlandet, hvilke puber er nærmest Guds hus?

In [13]:
# hvordan vet man riktig category? https://github.com/OvertureMaps/schema/blob/main/docs/schema/concepts/by-theme/places/overture_categories.csv

query_hent_puber = f"""
COPY(
  WITH places AS (
    SELECT
      id,
      names.primary AS name_primary,
      categories.primary AS category_primary,
      categories,
      confidence,
      websites,
      phones,
      brand.names.primary AS brand_name,
      geometry,
      ST_AsText(geometry) AS geometry_wkt,
      bbox
    FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*',
      hive_partitioning=true)
    WHERE bbox.xmin BETWEEN {min_x} AND {max_x}
      AND bbox.ymin BETWEEN {min_y} AND {max_y}
      AND categories.primary IN ('religious_organization', 'church_cathedral', 'evangelical_church', 'jehovahs_witness_kingdom_hall', 'pentecostal_church', 'church', 'place_of_worship', 'bar', 'pub', 'beer_hall', 'brewery', 'biergarten')
  ),
  pubs AS (
    SELECT * EXCLUDE(geometry), ST_Transform(geometry, 'EPSG:4236', 'EPSG:25833') AS geometry FROM places
    WHERE category_primary IN ('bar', 'pub', 'beer_hall', 'brewery', 'biergarten')
  ),
  churches AS (
    SELECT * EXCLUDE(geometry), ST_Transform(geometry, 'EPSG:4236', 'EPSG:25833') AS geometry FROM places
    WHERE category_primary IN ('religious_organization', 'church_cathedral', 'evangelical_church', 'jehovahs_witness_kingdom_hall', 'pentecostal_church', 'church', 'place_of_worship')
  )
  SELECT
    pub.id,
    pub.name_primary AS pub_name,
    pub.categories,
    pub.confidence,
    pub.websites,
    pub.phones,
    pub.brand_name,
    pub.geometry_wkt,
    pub.geometry,
    MIN(ST_Distance(pub.geometry, church.geometry)) AS distance_to_closest_church_meters,
    COUNT(church.id) AS nearby_church_count
  FROM pubs pub
  LEFT JOIN churches church
    ON ST_DWithin(pub.geometry, church.geometry, 1000) -- 1 km radius
  GROUP BY pub.id, pub.name_primary, pub.categories, pub.confidence, pub.websites, pub.phones, pub.brand_name, pub.geometry, pub.geometry_wkt
  ORDER BY pub.confidence DESC
) TO 'agder_puber.parquet' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 10_000)
"""

con.execute(query_hent_puber)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
# Vis resultat fra puber i Agder

con.sql("SELECT * FROM read_parquet('agder_puber.parquet') LIMIT 10").show()
con.sql("SELECT count(*) as antall_puber FROM read_parquet('agder_puber.parquet')").show()
con.sql("SELECT pub_name, confidence, distance_to_closest_church_meters, geometry_wkt FROM read_parquet('agder_puber.parquet') WHERE distance_to_closest_church_meters IS NOT NULL AND confidence > 0.7 ORDER BY distance_to_closest_church_meters ASC LIMIT 5").show()




# Prepare data for KeplerGL using GeoPandas
puber_df = con.sql("""
    SELECT 
        id, 
        pub_name, 
        categories, 
        confidence, 
        websites, 
        phones, 
        brand_name,
        ST_AsText(ST_Transform(geometry, 'EPSG:25833', 'EPSG:4236')) AS geometry_wkt,
        distance_to_closest_church_meters,
        nearby_church_count
    FROM read_parquet('agder_puber.parquet')
    WHERE distance_to_closest_church_meters IS NOT NULL
        AND confidence > 0.7
""").df()



puber_df = gpd.GeoDataFrame(
    puber_df, 
    geometry=gpd.GeoSeries.from_wkt(puber_df['geometry_wkt'].tolist()),
    crs="EPSG:4326"
)

puber_df = puber_df.drop(columns=['geometry_wkt'])
# Convert the distance column to float explicitly
#puber_df['distance_to_closest_church_meters'] = puber_df['distance_to_closest_church_meters']*1.0


# Kart med puber og kirker i Agder
map_puber = keplergl.KeplerGl(height=1000)


# Add data to the map  
map_puber.add_data(data=puber_df, name="Puber i nærheten av kirker i Agder")

map_puber

┌──────────────────────────────────┬───────────────────────────┬──────────────────────────────────────────────────────────────────────────────────┬────────────────────┬────────────────────────────────────────────────────────┬───────────────┬────────────┬──────────────────────────────┬──────────────────────────────────────────────┬───────────────────────────────────┬─────────────────────┐
│                id                │         pub_name          │                                    categories                                    │     confidence     │                        websites                        │    phones     │ brand_name │         geometry_wkt         │                   geometry                   │ distance_to_closest_church_meters │ nearby_church_count │
│             varchar              │          varchar          │                  struct("primary" varchar, alternate varchar[])                  │       double       │                       varchar[]                  

KeplerGl(data={'Puber i nærheten av kirker i Agder': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …